# Cartpole: REINFORCE Monte Carlo Policy Gradients

In this notebook we'll implement an agent <b>that plays Cartpole </b>

<img src="http://neuro-educator.com/wp-content/uploads/2017/09/DQN.gif" alt="Cartpole gif"/>


# This is a notebook from [Deep Reinforcement Learning Course with Tensorflow](https://simoninithomas.github.io/Deep_reinforcement_learning_Course/)
<img src="https://raw.githubusercontent.com/simoninithomas/Deep_reinforcement_learning_Course/master/docs/assets/img/DRLC%20Environments.png" alt="Deep Reinforcement Course"/>
<br>
<p>  Deep Reinforcement Learning Course is a free series of articles and videos tutorials 🆕 about Deep Reinforcement Learning, where **we'll learn the main algorithms (Q-learning, Deep Q Nets, Dueling Deep Q Nets, Policy Gradients, A2C, Proximal Policy Gradients…), and how to implement them with Tensorflow.**
<br><br>
    
📜The articles explain the architectures from the big picture to the mathematical details behind them.
<br>
📹 The videos explain how to build the agents with Tensorflow </b></p>
<br>
This course will give you a **solid foundation for understanding and implementing the future state of the art algorithms**. And, you'll build a strong professional portfolio by creating **agents that learn to play awesome environments**: Doom© 👹, Space invaders 👾, Outrun, Sonic the Hedgehog©, Michael Jackson’s Moonwalker, agents that will be able to navigate in 3D environments with DeepMindLab (Quake) and able to walk with Mujoco. 
<br><br>
</p> 

## 📚 The complete [Syllabus HERE](https://simoninithomas.github.io/Deep_reinforcement_learning_Course/)


## Any questions 👨‍💻
<p> If you have any questions, feel free to ask me: </p>
<p> 📧: <a href="mailto:hello@simoninithomas.com">hello@simoninithomas.com</a>  </p>
<p> Github: https://github.com/simoninithomas/Deep_reinforcement_learning_Course </p>
<p> 🌐 : https://simoninithomas.github.io/Deep_reinforcement_learning_Course/ </p>
<p> Twitter: <a href="https://twitter.com/ThomasSimonini">@ThomasSimonini</a> </p>
<p> Don't forget to <b> follow me on <a href="https://twitter.com/ThomasSimonini">twitter</a>, <a href="https://github.com/simoninithomas/Deep_reinforcement_learning_Course">github</a> and <a href="https://medium.com/@thomassimonini">Medium</a> to be alerted of the new articles that I publish </b></p>
    
## How to help  🙌
3 ways:
- **Clap our articles and like our videos a lot**:Clapping in Medium means that you really like our articles. And the more claps we have, the more our article is shared Liking our videos help them to be much more visible to the deep learning community.
- **Share and speak about our articles and videos**: By sharing our articles and videos you help us to spread the word. 
- **Improve our notebooks**: if you found a bug or **a better implementation** you can send a pull request.
<br>

## Important note 🤔
<b> You can run it on your computer but it's better to run it on GPU based services</b>, personally I use Microsoft Azure and their Deep Learning Virtual Machine (they offer 170$)
https://azuremarketplace.microsoft.com/en-us/marketplace/apps/microsoft-ads.dsvm-deep-learning
<br>
⚠️ I don't have any business relations with them. I just loved their excellent customer service.

If you have some troubles to use Microsoft Azure follow the explainations of this excellent article here (without last the part fast.ai): https://medium.com/@manikantayadunanda/setting-up-deeplearning-machine-and-fast-ai-on-azure-a22eb6bd6429

## Prerequisites 🏗️
Before diving on the notebook **you need to understand**:
- The foundations of Reinforcement learning (MC, TD, Rewards hypothesis...) [Article](https://medium.freecodecamp.org/an-introduction-to-reinforcement-learning-4339519de419)
- Policy gradients [Article](https://medium.freecodecamp.org/an-introduction-to-policy-gradients-with-cartpole-and-doom-495b5ef2207f)

## Step 1: Import the libraries 📚

In [1]:
import tensorflow as tf
import numpy as np
import gym

## Step 2: Create our environment 🎮
This time we use <a href="https://gym.openai.com/">OpenAI Gym</a> which has a lot of great environments.

In [2]:
env = gym.make('CartPole-v0')
env = env.unwrapped
# Policy gradient has high variance, seed for reproducability
env.seed(1)

[1]

## Step 3: Set up our hyperparameters ⚗️

In [3]:
## ENVIRONMENT Hyperparameters
state_size = 4
action_size = env.action_space.n

## TRAINING Hyperparameters
max_episodes = 300
learning_rate = 0.01
gamma = 0.95 # Discount rate

## Step 4 : Define the preprocessing functions ⚙️
This function takes <b>the rewards and perform discounting.</b>

In [4]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * gamma + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)
    
    return discounted_episode_rewards

## Step 5: Create our Policy Gradient Neural Network model 🧠

<img src="https://raw.githubusercontent.com/simoninithomas/Deep_reinforcement_learning_Course/master/Policy%20Gradients/Cartpole/assets/catpole.png">

The idea is simple:
- Our state which is an array of 4 values will be used as an input.
- Our NN is 3 fully connected layers.
- Our output activation function is softmax that squashes the outputs to a probability distribution (for instance if we have 4, 2, 6 --> softmax --> (0.4, 0.2, 0.6)

In [5]:
with tf.name_scope("inputs"):
    input_ = tf.placeholder(tf.float32, [None, state_size], name="input_")
    actions = tf.placeholder(tf.int32, [None, action_size], name="actions")
    discounted_episode_rewards_ = tf.placeholder(tf.float32, [None,], name="discounted_episode_rewards")
    
    # Add this placeholder for having this variable in tensorboard
    mean_reward_ = tf.placeholder(tf.float32 , name="mean_reward")

    with tf.name_scope("fc1"):
        fc1 = tf.contrib.layers.fully_connected(inputs = input_,
                                                num_outputs = 10,
                                                activation_fn=tf.nn.relu,
                                                weights_initializer=tf.contrib.layers.xavier_initializer())

    with tf.name_scope("fc2"):
        fc2 = tf.contrib.layers.fully_connected(inputs = fc1,
                                                num_outputs = action_size,
                                                activation_fn= tf.nn.relu,
                                                weights_initializer=tf.contrib.layers.xavier_initializer())
    
    with tf.name_scope("fc3"):
        fc3 = tf.contrib.layers.fully_connected(inputs = fc2,
                                                num_outputs = action_size,
                                                activation_fn= None,
                                                weights_initializer=tf.contrib.layers.xavier_initializer())

    with tf.name_scope("softmax"):
        action_distribution = tf.nn.softmax(fc3)

    with tf.name_scope("loss"):
        # tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the result after applying the softmax function
        # If you have single-class labels, where an object can only belong to one class, you might now consider using 
        # tf.nn.sparse_softmax_cross_entropy_with_logits so that you don't have to convert your labels to a dense one-hot array. 
        neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = fc3, labels = actions)
        loss = tf.reduce_mean(neg_log_prob * discounted_episode_rewards_) 
        
    
    with tf.name_scope("train"):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.


## Step 6: Set up Tensorboard 📊
For more information about tensorboard, please watch this <a href="https://www.youtube.com/embed/eBbEDRsCmv4">excellent 30min tutorial</a> <br><br>
To launch tensorboard : `tensorboard --logdir=./tensorboard/pg/1`

In [6]:
# Setup TensorBoard Writer
writer = tf.summary.FileWriter("./tensorboard/pg/1")

## Losses
tf.summary.scalar("Loss", loss)

## Reward mean
tf.summary.scalar("Reward_mean", mean_reward_)

write_op = tf.summary.merge_all()

## Step 7: Train our Agent 🏃‍♂️

Create the NN
maxReward = 0 # Keep track of maximum reward
For episode in range(max_episodes):
    episode + 1
    reset environment
    reset stores (states, actions, rewards)
    
    For each step:
        Choose action a
        Perform action a
        Store s, a, r
        If done:
            Calculate sum reward
            Calculate gamma Gt
            Optimize

In [7]:
allRewards = []
total_rewards = 0
maximumRewardRecorded = 0
episode = 0
episode_states, episode_actions, episode_rewards = [],[],[]

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for episode in range(max_episodes):
        
        episode_rewards_sum = 0

        # Launch the game
        state = env.reset()
        
        env.render()
           
        while True:
            
            # Choose action a, remember WE'RE NOT IN A DETERMINISTIC ENVIRONMENT, WE'RE OUTPUT PROBABILITIES.
            action_probability_distribution = sess.run(action_distribution, feed_dict={input_: state.reshape([1,4])})
            
            action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob

            # Perform a
            new_state, reward, done, info = env.step(action)

            # Store s, a, r
            episode_states.append(state)
                        
            # For actions because we output only one (the index) we need 2 (1 is for the action taken)
            # We need [0., 1.] (if we take right) not just the index
            action_ = np.zeros(action_size)
            action_[action] = 1
            
            episode_actions.append(action_)
            
            episode_rewards.append(reward)
            if done:
                # Calculate sum reward
                episode_rewards_sum = np.sum(episode_rewards)
                
                allRewards.append(episode_rewards_sum)
                
                total_rewards = np.sum(allRewards)
                
                # Mean reward
                mean_reward = np.divide(total_rewards, episode+1)
                
                
                maximumRewardRecorded = np.amax(allRewards)
                
                print("==========================================")
                print("Episode: ", episode)
                print("Reward: ", episode_rewards_sum)
                print("Mean Reward", mean_reward)
                print("Max reward so far: ", maximumRewardRecorded)
                
                # Calculate discounted reward
                discounted_episode_rewards = discount_and_normalize_rewards(episode_rewards)
                                
                # Feedforward, gradient and backpropagation
                loss_, _ = sess.run([loss, train_opt], feed_dict={input_: np.vstack(np.array(episode_states)),
                                                                 actions: np.vstack(np.array(episode_actions)),
                                                                 discounted_episode_rewards_: discounted_episode_rewards 
                                                                })
                
 
                                                                 
                # Write TF Summaries
                summary = sess.run(write_op, feed_dict={input_: np.vstack(np.array(episode_states)),
                                                                 actions: np.vstack(np.array(episode_actions)),
                                                                 discounted_episode_rewards_: discounted_episode_rewards,
                                                                    mean_reward_: mean_reward
                                                                })
                
               
                writer.add_summary(summary, episode)
                writer.flush()
                
            
                
                # Reset the transition stores
                episode_states, episode_actions, episode_rewards = [],[],[]
                
                break
            
            state = new_state
        
        # Save Model
        if episode % 100 == 0:
            saver.save(sess, "./models/model.ckpt")
            print("Model saved")

Episode:  0
Reward:  63.0
Mean Reward 63.0
Max reward so far:  63.0
Model saved
Episode:  1
Reward:  37.0
Mean Reward 50.0
Max reward so far:  63.0
Episode:  2
Reward:  9.0
Mean Reward 36.333333333333336
Max reward so far:  63.0
Episode:  3
Reward:  15.0
Mean Reward 31.0
Max reward so far:  63.0
Episode:  4
Reward:  49.0
Mean Reward 34.6
Max reward so far:  63.0
Episode:  5
Reward:  64.0
Mean Reward 39.5
Max reward so far:  64.0
Episode:  6
Reward:  46.0
Mean Reward 40.42857142857143
Max reward so far:  64.0
Episode:  7
Reward:  18.0
Mean Reward 37.625
Max reward so far:  64.0
Episode:  8
Reward:  13.0
Mean Reward 34.888888888888886
Max reward so far:  64.0
Episode:  9
Reward:  20.0
Mean Reward 33.4
Max reward so far:  64.0
Episode:  10
Reward:  33.0
Mean Reward 33.36363636363637
Max reward so far:  64.0
Episode:  11
Reward:  12.0
Mean Reward 31.583333333333332
Max reward so far:  64.0
Episode:  12
Reward:  12.0
Mean Reward 30.076923076923077
Max reward so far:  64.0
Episode:  13
Rewar

Episode:  69
Reward:  24.0
Mean Reward 27.214285714285715
Max reward so far:  100.0
Episode:  70
Reward:  24.0
Mean Reward 27.169014084507044
Max reward so far:  100.0
Episode:  71
Reward:  35.0
Mean Reward 27.27777777777778
Max reward so far:  100.0
Episode:  72
Reward:  13.0
Mean Reward 27.08219178082192
Max reward so far:  100.0
Episode:  73
Reward:  9.0
Mean Reward 26.83783783783784
Max reward so far:  100.0
Episode:  74
Reward:  50.0
Mean Reward 27.14666666666667
Max reward so far:  100.0
Episode:  75
Reward:  10.0
Mean Reward 26.92105263157895
Max reward so far:  100.0
Episode:  76
Reward:  111.0
Mean Reward 28.01298701298701
Max reward so far:  111.0
Episode:  77
Reward:  26.0
Mean Reward 27.987179487179485
Max reward so far:  111.0
Episode:  78
Reward:  18.0
Mean Reward 27.860759493670887
Max reward so far:  111.0
Episode:  79
Reward:  37.0
Mean Reward 27.975
Max reward so far:  111.0
Episode:  80
Reward:  31.0
Mean Reward 28.012345679012345
Max reward so far:  111.0
Episode:  

Episode:  135
Reward:  38.0
Mean Reward 27.779411764705884
Max reward so far:  111.0
Episode:  136
Reward:  33.0
Mean Reward 27.817518248175183
Max reward so far:  111.0
Episode:  137
Reward:  33.0
Mean Reward 27.855072463768117
Max reward so far:  111.0
Episode:  138
Reward:  20.0
Mean Reward 27.798561151079138
Max reward so far:  111.0
Episode:  139
Reward:  46.0
Mean Reward 27.928571428571427
Max reward so far:  111.0
Episode:  140
Reward:  29.0
Mean Reward 27.93617021276596
Max reward so far:  111.0
Episode:  141
Reward:  28.0
Mean Reward 27.93661971830986
Max reward so far:  111.0
Episode:  142
Reward:  43.0
Mean Reward 28.041958041958043
Max reward so far:  111.0
Episode:  143
Reward:  56.0
Mean Reward 28.23611111111111
Max reward so far:  111.0
Episode:  144
Reward:  14.0
Mean Reward 28.137931034482758
Max reward so far:  111.0
Episode:  145
Reward:  20.0
Mean Reward 28.08219178082192
Max reward so far:  111.0
Episode:  146
Reward:  127.0
Mean Reward 28.755102040816325
Max rewar

Episode:  207
Reward:  69.0
Mean Reward 29.95673076923077
Max reward so far:  127.0
Episode:  208
Reward:  63.0
Mean Reward 30.114832535885167
Max reward so far:  127.0
Episode:  209
Reward:  13.0
Mean Reward 30.033333333333335
Max reward so far:  127.0
Episode:  210
Reward:  18.0
Mean Reward 29.976303317535546
Max reward so far:  127.0
Episode:  211
Reward:  20.0
Mean Reward 29.92924528301887
Max reward so far:  127.0
Episode:  212
Reward:  34.0
Mean Reward 29.948356807511736
Max reward so far:  127.0
Episode:  213
Reward:  27.0
Mean Reward 29.934579439252335
Max reward so far:  127.0
Episode:  214
Reward:  39.0
Mean Reward 29.976744186046513
Max reward so far:  127.0
Episode:  215
Reward:  74.0
Mean Reward 30.180555555555557
Max reward so far:  127.0
Episode:  216
Reward:  22.0
Mean Reward 30.142857142857142
Max reward so far:  127.0
Episode:  217
Reward:  41.0
Mean Reward 30.192660550458715
Max reward so far:  127.0
Episode:  218
Reward:  39.0
Mean Reward 30.232876712328768
Max rewa

Episode:  274
Reward:  115.0
Mean Reward 40.01090909090909
Max reward so far:  316.0
Episode:  275
Reward:  189.0
Mean Reward 40.55072463768116
Max reward so far:  316.0
Episode:  276
Reward:  152.0
Mean Reward 40.95306859205776
Max reward so far:  316.0
Episode:  277
Reward:  452.0
Mean Reward 42.431654676258994
Max reward so far:  452.0
Episode:  278
Reward:  173.0
Mean Reward 42.89964157706093
Max reward so far:  452.0
Episode:  279
Reward:  46.0
Mean Reward 42.910714285714285
Max reward so far:  452.0
Episode:  280
Reward:  778.0
Mean Reward 45.52669039145908
Max reward so far:  778.0
Episode:  281
Reward:  425.0
Mean Reward 46.87234042553192
Max reward so far:  778.0
Episode:  282
Reward:  149.0
Mean Reward 47.23321554770318
Max reward so far:  778.0
Episode:  283
Reward:  100.0
Mean Reward 47.41901408450704
Max reward so far:  778.0
Episode:  284
Reward:  87.0
Mean Reward 47.55789473684211
Max reward so far:  778.0
Episode:  285
Reward:  72.0
Mean Reward 47.64335664335665
Max rew

In [9]:
with tf.Session() as sess:
    env.reset()
    rewards = []
    
    # Load the model
    saver.restore(sess, "./models/model.ckpt")

    for episode in range(10):
        state = env.reset()
        step = 0
        done = False
        total_rewards = 0
        print("****************************************************")
        print("EPISODE ", episode)

        while True:
            

            # Choose action a, remember WE'RE NOT IN A DETERMINISTIC ENVIRONMENT, WE'RE OUTPUT PROBABILITIES.
            action_probability_distribution = sess.run(action_distribution, feed_dict={input_: state.reshape([1,4])})
            #print(action_probability_distribution)
            action = np.random.choice(range(action_probability_distribution.shape[1]), p=action_probability_distribution.ravel())  # select action w.r.t the actions prob


            new_state, reward, done, info = env.step(action)

            total_rewards += reward

            if done:
                rewards.append(total_rewards)
                print ("Score", total_rewards)
                break
            state = new_state
    env.close()
    print ("Score over time: " +  str(sum(rewards)/10))

INFO:tensorflow:Restoring parameters from ./models/model.ckpt
****************************************************
EPISODE  0
Score 20.0
****************************************************
EPISODE  1
Score 18.0
****************************************************
EPISODE  2
Score 28.0
****************************************************
EPISODE  3
Score 19.0
****************************************************
EPISODE  4
Score 13.0
****************************************************
EPISODE  5
Score 46.0
****************************************************
EPISODE  6
Score 55.0
****************************************************
EPISODE  7
Score 16.0
****************************************************
EPISODE  8
Score 17.0
****************************************************
EPISODE  9
Score 9.0
Score over time: 24.1
